In [1]:
import os
from dotenv import load_dotenv
#from elasticsearch import Elasticsearch, ConnectionError

#es_client = Elasticsearch('http://localhost:9200')


# Load environment variables from the .env file
load_dotenv()
os.environ['ROOT_DATA_DIR']

'/Users/username/PycharmProjects/leaf-buddy/data'

In [2]:
from IPython.display import clear_output

!rm gsheets.py
!wget https://gist.githubusercontent.com/aleksandr-dzhumurat/c1b52a6284fab8e8355ef25cc3a3cc51/raw/2e0ef91fc111e7b838594dd01a309bd466dbafde/gsheets.py
clear_output()
print('Pakages installed')

Pakages installed


In [3]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
clear_output()

In [4]:
# from gsheets import get_spreadsheet, duplicate_tab, insert_data_dummy

# new_tab_name = 'leafly_catalog'

# g_sheet_link = 'https://docs.google.com/spreadsheets/d/1cOZ930D8KVupQQciC_bUOXqrRlVytKhs5tXlDKjpLaY/edit?gid=0#gid=0'
# spreadsheet_id = g_sheet_link.split('/')[-2]
# print(spreadsheet_id)
# spreadsheet = get_spreadsheet(spreadsheet_id)
# duplicate_tab(spreadsheet, 'Sheet1', new_tab_name)

# worksheet = spreadsheet.worksheet(new_tab_name)

# insert_data_dummy(worksheet, search_results_df, ascending=[False, True])

In [5]:
import os
import pandas as pd

data_version = '7268cbe9f2b1'
catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], f'leafly_catalog_{data_version}.csv'))
print(catalog_df.shape[0])
catalog_df.head(3)

49259


,doc_id,category,title,link,file,processed_file_name
0,doc_c2ccd832-edc6-4533-aa95-2b83997b6235,cannabis/flower,3.5g THC-A COOKIES PINK RUNTZ FLOWER HYBRID,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...
1,doc_0f4e3e88-e015-4358-a160-cf2b202dcf75,cannabis/flower,7.5 g THC-A Pink Rozay Hybrid,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...
2,doc_da5a1680-918e-4d80-b6cc-cd587edf90a1,cannabis/flower,LEMONADE BLANCO 1 G FLOWER THC,/brands/aventus-8/products/aventus-8-torch-thc...,/Users/username/PycharmProjects/leaf-buddy/dat...,/Users/username/PycharmProjects/leaf-buddy/dat...


In [6]:
sweed_catalog_df = None
if sweed_catalog_df is None:
    sweed_catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'products_dataset_ffb59da3.csv.gzip'))
print(f"Catalog length {sweed_catalog_df.shape[0]}, {sweed_catalog_df['ProductCategory'].unique()}")
sweed_catalog_df.head(5)                               

Catalog length 1513, ['Accessories Temp' 'Concentrates' 'Vapes' 'Apparel' 'Pre-Rolls' 'Flower'
 'Fees' 'Edibles' 'Oral Products' 'Accessories'
 'Accessories - Cabbage Club' 'Topicals' 'Temp Accessories'
 'Apparel - Cabbage Club']


,DealerID,ProductID,ProductName,ProductCategory,ProductCategoryID,ProductTypeID,ProductType,ProductBrand,ProductGroupName,ProductClass,ProductStrainPrevalence,ProductSubCategory,SizeName,ProductPrice,product_category,pre_size_name,size_name
0,211388,190856,Test Accessory EACH,Accessories Temp,3459,0,NaN,No Brand,Test Accessory,Non-cannabis,N/S,NaN,EACH,100,accessories temp,each,each
1,211388,263581,Grape Noir CDT Dablicator 1g,Concentrates,3483,0,NaN,MÜV,Grape Noir CDT Dablicator,Cannabis,Indica,CDT Dablicator,1g,60,concentrates,1g,1g
2,211388,263599,Layered Cake Badder 1g,Concentrates,3483,0,NaN,MÜV,Layered Cake Badder,Cannabis,Indica,Live Resin Badder,1g,60,concentrates,1g,1g
3,211388,263822,Layered Cake 0.5g,Vapes,3484,0,NaN,MÜV,Layered Cake,Cannabis,Indica,Live Resin Cartridge,0.5g,50,vape,0.5g,0.5g
4,211388,264161,Triangle Kush CDT Dablicator 1g,Concentrates,3483,0,NaN,MÜV,Triangle Kush CDT Dablicator,Cannabis,Indica,CDT Dablicator,1g,60,concentrates,1g,1g


In [7]:
sales_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'sales_dataset_ffb59da3.csv.gzip'))
# print(sales_df.shape[0])
# sweed_catalog_df[
#     (sweed_catalog_df['ProductID'].isin(.index[:100]))
#     # & (~sweed_catalog_df['ProductBrand'].isin(['(the) Essence', 'Bits']))
#     & (~sweed_catalog_df['ProductCategory'].isin(['Vapes']))
# ]
product_sales_df = (
    sales_df
    .groupby('ProductID')['CustomerID'].count()
    .reset_index(name='purchase_count').sort_values(by='purchase_count', ascending=False)
)
product_sales_df.head(3)

,ProductID,purchase_count
31,192981,12098
37,192987,8597
33,192983,7437


In [8]:
import os
import pandas as pd

leafly_search_results = '/Users/username/PycharmProjects/leaf-buddy/data/search/search_e6da18c153cb'


matched_catalog = (
         pd.concat([
            pd.read_csv(os.path.join(leafly_search_results, i))
            for i in os.listdir(leafly_search_results) if '.csv' in i
        ])
 )
search_results_df = (
    product_sales_df
    .merge(sweed_catalog_df[['ProductID', 'ProductName', 'ProductCategory', 'ProductBrand']], on='ProductID')
    .merge(matched_catalog.rename(columns={'sweed_product_id': 'ProductID', 'purchase_count': 'sweed_purchases'}), on='ProductID')
    .sort_values(by='purchase_count', ascending=False)
)
print(search_results_df.shape[0])
search_results_df.to_csv(os.path.join('/Users/username/PycharmProjects/leaf-buddy/data/search', 'sweed_leafly_mapping.csv'), index=False)
search_link_dedupled_df = search_results_df[['title', 'link']].groupby('title').agg(lambda x: x.iloc[0]).reset_index()
search_results_df.head()

10735


,ProductID,purchase_count,ProductName,ProductCategory,ProductBrand,sweed_product_name,title,doc_id,brand,link,item_url,price,uom
0,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE RSO | 10 PACK,doc_7efafe668209,Verano,/dispensary-info/cloud-nine-dispensary/p/63214...,https://www.leafly.com/dispensary-info/cloud-n...,$32.22,each
8,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],100mg Concord Grape RSO,doc_63481b638580,Verano,/dispensary-info/brute-s-roots/p/627740137/100...,https://www.leafly.com/dispensary-info/brute-s...,$29.00,each
1,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE - 10 PACK - RSO SOFT LOZENGES - ...,doc_9c516ef2fa58,Encore Edibles,/dispensary-info/elevated-herb-nj/p/637147171/...,https://www.leafly.com/dispensary-info/elevate...,$28.50,each
14,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO Gummies 100mg,doc_ae339d8bbd89,Encore Edibles,/dispensary-info/zen-leaf-las-vegas-flamingo-r...,https://www.leafly.com/dispensary-info/zen-lea...,$24.00,each
13,192981,12098,Concord Grape RSO Soft Chews 100mg [10-Pack],Edibles,Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO Gummies 100mg,doc_1a8fba353660,Encore Edibles,/dispensary-info/zen-leaf-north-las-vegas/p/63...,https://www.leafly.com/dispensary-info/zen-lea...,$24.00,each


In [9]:
mapping_df = search_results_df[['ProductID', 'ProductName', 'ProductBrand', 'sweed_product_name', 'title', 'brand', 'doc_id']].drop_duplicates()

mapping_df.head()

,ProductID,ProductName,ProductBrand,sweed_product_name,title,brand
0,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE RSO | 10 PACK,Verano
8,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],100mg Concord Grape RSO,Verano
1,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE - 10 PACK - RSO SOFT LOZENGES - ...,Encore Edibles
14,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO Gummies 100mg,Encore Edibles
10,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Encore,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO - 10 Pack Edibles - Adult Use,Verano


In [10]:
import requests
import json
import openai

def create_matcher(api_key=None, model="gpt-3.5-turbo"):
    openai_api_key = api_key or os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise ValueError("OpenAI API key must be provided or set as OPENAI_API_KEY environment variable")
    system_message ="""
        You are a professional product matching expert for comparing items across different catalogs. Follow these rules to find the best match:
        
        * Match by brand (most important). If brand not matched it is most probably mismatch
        * Match by product name.
        * Match by units of measure.
        
        Input format:
        
        You will receive one source product (name and brand).
        After the source product, you will get a numbered list of target products.
        Matching rules:
        
        Prioritize longer product names if multiple matches have similar confidence.
        If the brand doesn't match, the confidence should be low unless other factors strongly align.
        If no clear match is found or you are unsure, return "ERROR" (no structured output).
        Output format:
        Return only structured output in this format:
        
        {"best_match": "product_number", "confidence": "confidence_level"}
        
        Confidence levels:
        "high": Strong match across brand, product name, and unit.
        "medium": Good match but with some uncertainty (e.g., minor name differences).
        "low": Weak match, possibly only on product name or unit.
        Important:
        
        Do not include explanations.
        Focus on brand importance and prefer longer names for better matches.
    """
    def match_item(user_prompt):
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {openai_api_key}"
        }
        
        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt}
            ],
            "temperature": 0.1  # Low temperature for consistent translations
        }
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code == 200:
            result = response.json()
            matched_items = result["choices"][0]["message"]["content"].strip()
            return matched_items
        else:
            error_message = f"API Error {response.status_code}: {response.text}"
            raise Exception(error_message)
    
    return match_item

matcher = create_matcher()

In [11]:
import backoff
from json.decoder import JSONDecodeError

def prepare_prompt(df_dict):
    res = 'source pruduct: product name: {}, product brand:  {}\n\n'.format(df_dict[0]['sweed_product_name'], df_dict[0]['ProductBrand'])
    for i in df_dict:
        res += '{}: product name: {}, product brand: {}\n'.format(i['row_num'], i['title'], i['brand'])
    return res

@backoff.on_exception(backoff.expo, JSONDecodeError, max_tries=3)
def find_match(sub_df):
    prompt = prepare_prompt(sub_df.to_dict(orient='records'))
    llm_resp = matcher(prompt)
    resp = json.loads(llm_resp)
    sub_df['match_confidence'] = sub_df['row_num'].apply(lambda x: resp['confidence'] if resp['best_match'] == str(x) else None)
    return sub_df

print('functions created')

functions created


In [12]:

# def find_match(sub_df):
#     prompt = prepare_prompt(sub_df.to_dict(orient='records'))
#     llm_resp = matcher(prompt)
#     print(llm_resp, type(llm_resp))
#     resp = json.loads(llm_resp)
#     sub_df['match_confidence'] = sub_df['row_num'].apply(lambda x: resp['confidence'] if resp['best_match'] == str(x) else None)
#     return sub_df

product_id = 193467

sub_df = mapping_df[mapping_df['ProductID'] == product_id].copy()
sub_df['row_num'] = sub_df.reset_index().index + 1
prompt = prepare_prompt(sub_df.to_dict(orient='records'))
llm_resp = matcher(prompt)
print(llm_resp, type(llm_resp))
# resp = json.loads(llm_resp)
sub_df.head()

{"best_match": "3", "confidence": "high"} <class 'str'>


,ProductID,ProductName,ProductBrand,sweed_product_name,title,brand,row_num
1222,193467,Georgia Pie Dart 1g,Sweet Supply,Georgia Pie Dart 1g,Georgia Pie,Revolution Cannabis,1
1229,193467,Georgia Pie Dart 1g,Sweet Supply,Georgia Pie Dart 1g,Georgia Pie [1g],Simply Herb,2
1224,193467,Georgia Pie Dart 1g,Sweet Supply,Georgia Pie Dart 1g,Georgia Pie,Georgia Pie,3
1223,193467,Georgia Pie Dart 1g,Sweet Supply,Georgia Pie Dart 1g,Georgia Pie [1g],Bodega Buds,4
1221,193467,Georgia Pie Dart 1g,Sweet Supply,Georgia Pie Dart 1g,Georgia Pie - 1g,Flower of Life,5


In [16]:
import random


products_list = mapping_df['ProductID'].unique().tolist()

def prepare_matching(output_dir):
    for i in products_list:
        output_file_name = f'{i}.csv'
        output_file = os.path.join(output_dir, output_file_name)
        if not os.path.exists(output_file):
            sub_df = mapping_df[mapping_df['ProductID'] == i].copy()
            sub_df['row_num'] = sub_df.reset_index().index + 1
            sub_df = find_match(sub_df)
            sub_df.to_csv(output_file, index=False)
        else:
            # print(f'File exists: {output_file_name}')
            pass
    print(f'All done! Num files: {len(os.listdir(output_dir))}')

output_dir = os.path.join(os.environ['ROOT_DATA_DIR'], 'pipelines-data', 'matched_data')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
file_list = os.listdir(output_dir)
print(len(file_list))

718


In [21]:
final_df = pd.concat([pd.read_csv(os.path.join(output_dir, i)) for i in os.listdir(output_dir)])
final_df.head()

,ProductID,ProductName,ProductBrand,sweed_product_name,title,brand,row_num,match_confidence
0,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,500mg Dart Pod | Banana Kush | INSA,INSA,1,NaN
1,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,MANGO KUSH 0.5G DISTILLATE SYRINGE,Goldsmith Extracts,2,NaN
2,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,"Mango Kush 0.5g, Live Rosin",TRENDI,3,high
3,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,Purple Kush RSO+ Live Resin Dart,Rise x,4,NaN
4,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,Bubba Kush Dart Pod,Moxie Extracts,5,NaN


In [32]:
(
    final_df[['ProductID', 'ProductName', 'ProductBrand', 'title', 'brand', 'match_confidence']]
    .merge(search_link_dedupled_df, on='title')
    .to_csv(os.path.join(os.environ['ROOT_DATA_DIR'], 'matched_leafly_sweed_dataset.csv'), index=False)
)

print('data saved')

data saved


,ProductID,ProductName,ProductBrand,sweed_product_name,title,brand,row_num,match_confidence
2,193484,Mango Kush Dart 0.5g,Sweet Supply,Mango Kush Dart 0.5g,"Mango Kush 0.5g, Live Rosin",TRENDI,3,high
8,210053,G-Wagon Budder 1g,Sweet Supply,G-Wagon Budder 1g,G-Wagon 3.5g,Victoria Cannabis Company,9,high
1,280332,Tropical Teeth #8 0.3g,(the) Essence,Tropical Teeth #8 0.3g,Tiramisu Teeth 3.5g,(the) Essence,2,high
1,193490,Mint Shake Shatter 1g,Sweet Supply,Mint Shake Shatter 1g,Mint Kush Shatter 1g Cured Extract,Roll One,2,medium
1,193309,Triangle Kush 1g,MÜV,Triangle Kush 1g,Triangle Kush,Verano,2,low
...,...,...,...,...,...,...,...,...
1,193310,Triangle Kush CDT Syringe 1g,MÜV,Triangle Kush CDT Syringe 1g,Triangle Kush,Verano,2,high
2,193462,Fruit Pops 0.7g [1-Pack],Sweet Supply,Fruit Pops 0.7g [1-Pack],Fruit Pops 3.5g,(the) Essence,3,medium
4,235590,Orange Push Pop 0.3g,MÜV,Orange Push Pop 0.3g,Orange Push Pop,Verano,5,medium
3,193489,Mint Shake Crumble 1g,Sweet Supply,Mint Shake Crumble 1g,Primal Animal Mint Crumble (H) 1g,Primal Extracts,4,high


In [35]:
output_path = os.path.join(os.environ['ROOT_DATA_DIR'], 'app_matched_leafly_sweed_dataset.csv')

(
    final_df[~final_df['match_confidence'].isna()][['ProductID', 'ProductName', 'ProductBrand', 'title', 'brand', 'match_confidence']]
    .merge(search_link_dedupled_df, on='title')
    .to_csv(output_path, index=False)
)

print(f'data saved to {output_path}')

data saved to /Users/username/PycharmProjects/leaf-buddy/data/app_matched_leafly_sweed_dataset.csv


In [35]:
matching_dataset_df = search_results_df[['ProductID', 'sweed_product_name', 'title', 'brand']].drop_duplicates()

print(matching_dataset_df.shape[0])
matching_dataset_df.head()

5155


,ProductID,sweed_product_name,title,brand
0,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE RSO | 10 PACK,Verano
8,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],100mg Concord Grape RSO,Verano
1,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],CONCORD GRAPE - 10 PACK - RSO SOFT LOZENGES - ...,Encore Edibles
14,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO Gummies 100mg,Encore Edibles
10,192981,Concord Grape RSO Soft Chews 100mg [10-Pack],Concord Grape RSO - 10 Pack Edibles - Adult Use,Verano


In [7]:
from utils import BM25Index

index = BM25Index()
corpus = catalog_df['title'].values.tolist()
print(f'corpus len: {len(corpus)}')
index.build_index(corpus)

corpus len: 49259
num documents: 49259


In [8]:
index.query('B-52 Bomber')

[]

In [29]:
sweed_catalog_df[
    (sweed_catalog_df['ProductName'].apply(lambda x: 'mint' in x))
    # & (catalog_df['title'].apply(lambda x: 'noir' in x.lower()))
]

,DealerID,ProductID,ProductName,ProductCategory,ProductCategoryID,ProductTypeID,ProductType,ProductBrand,ProductGroupName,ProductClass,ProductStrainPrevalence,ProductSubCategory,SizeName,ProductPrice,product_category,pre_size_name,size_name
642,211388,193216,Peppermint Dark Chocolates 100mg [10-Pack],Edibles,3488,0,NaN,MÜV,Peppermint Dark Chocolates,Cannabis,N/S,Chocolates,10mg,20,edibles,10mg,10mg
758,211388,193332,Wintermint Meltaways 100mg [20-Pack],Edibles,3488,0,NaN,MÜV,Wintermint Meltaways,Cannabis,N/S,Mints,5mg,20,edibles,5mg,5mg


In [32]:
catalog_df[
    # (catalog_df['link'].apply(lambda x: '/brands/verano' in x))
    (catalog_df['title'].apply(lambda x: 'bonsay' in x.lower()))
]

,doc_id,category,title,link,file,processed_file_name


In [9]:
import math
from collections import Counter, defaultdict

def tokenize(text):
    return text.lower().split()

class OptimizedBM25Index:
    """
    Optimized BM25 implementation for large document collections
    
    Usage:
        index = OptimizedBM25Index()
        index.build_index(corpus)
        results = index.query("your query here")
    """
    def __init__(self, k1=1.5, b=0.75):
        self.k1 = k1
        self.b = b
        self.idf_scores = {}
        self.term_document_freq = defaultdict(dict)
        self.doc_lengths = []
        self.avg_doc_len = 0
        self.corpus_size = 0
        
    def build_index(self, corpus):
        print(f'num documents: {len(corpus)}')
        self.corpus_size = len(corpus)
        self.doc_lengths = []
        total_tokens = 0
        doc_freqs = defaultdict(int)
        for doc_id, doc in enumerate(corpus):
            tokens = tokenize(doc)
            doc_len = len(tokens)
            self.doc_lengths.append(doc_len)
            total_tokens += doc_len
            term_freqs = Counter(tokens)
            for term, freq in term_freqs.items():
                self.term_document_freq[term][doc_id] = freq
                doc_freqs[term] += 1
        self.avg_doc_len = total_tokens / self.corpus_size
        for term, freq in doc_freqs.items():
            self.idf_scores[term] = math.log((self.corpus_size - freq + 0.5) / (freq + 0.5) + 1.0)
        
        print('Index built successfully')

    def query(self, query, top_k=None):
        query_terms = tokenize(query)
        scores = defaultdict(float)
        candidate_docs = set()
        for term in query_terms:
            if term in self.term_document_freq:
                candidate_docs.update(self.term_document_freq[term].keys())
        for doc_id in candidate_docs:
            doc_len = self.doc_lengths[doc_id]
            doc_len_norm = doc_len / self.avg_doc_len
            
            for term in query_terms:
                if term in self.idf_scores and doc_id in self.term_document_freq.get(term, {}):
                    tf = self.term_document_freq[term][doc_id]
                    idf = self.idf_scores[term]
                    
                    numerator = tf * (self.k1 + 1)
                    denominator = tf + self.k1 * (1 - self.b + self.b * doc_len_norm)
                    scores[doc_id] += idf * (numerator / denominator)
        
        results = [(doc_id, score) for doc_id, score in scores.items()]
        results.sort(key=lambda x: -x[1])
        if top_k:
            return results[:top_k]
        return results
    
    def batch_query(self, queries, top_k=None):
        """
        Process multiple queries at once
        """
        results = []
        for query in queries:
            results.append(self.query(query, top_k=top_k))
        return results

index = OptimizedBM25Index()
index.build_index(corpus)

# Single query
results = index.query("Bomber", top_k=10)


num documents: 49259
Index built successfully


In [12]:
results = index.query("bomber", top_k=10)

results

[]

In [111]:
# sweed_catalog_df[sweed_catalog_df['ProductName']==random_name]
# print(sweed_catalog_df.shape[0])
# sweed_catalog_df[sweed_catalog_df['ProductName'].apply(lambda x: 'slurricane' in x.lower())]

In [86]:
verano_data_version = 'ee72f7351d78'
verano_catalog_df = pd.read_csv(os.path.join(os.environ['ROOT_DATA_DIR'], f'verano_leafly_catalog_{verano_data_version}.csv'))
print(verano_catalog_df.shape[0])
verano_catalog_df[verano_catalog_df['title'].apply(lambda x: 'layered' in x.lower())]

404


,doc_id,category,title,link,file,processed_file_name


In [74]:
random_name = sweed_catalog_df[sweed_catalog_df['ProductCategory']== 'Pre-Rolls'].sort_values('ProductName')['ProductName'].sample(1).iloc[0]
print(random_name)
#catalog_df.iloc[[i[0] for i in index.query(random_name)[:5]]]

Punch Cookies 3.5g [5-Pack]


In [47]:
from time import sleep
from IPython.display import clear_output

def monitor_scraping(data_version):
    prev = None
    while True:
        num_files = len(os.listdir(f'/Users/username/PycharmProjects/leaf-buddy/data/leafly_html_pages_{data_version}/items_html_pages_{data_version}'))
        if prev is None:
            prev = num_files
        else:
            if num_files == prev:
                raise RuntimeError(f'Scrapping was stucked, num files: {num_files}')
            prev = num_files
        print(f'num files {num_files}')
        sleep(10)
        clear_output()


        
        # monitor_scraping(data_version='7268cbe9f2b1')

In [52]:
import requests

from IPython.display import display, HTML

api_url = 'http://0.0.0.0:8000'

def request_api(query, relief, positive_effects):
    headers = {"Content-Type": "application/json"}
    payload = {
        "relief": relief,
        "positive_effects": positive_effects,
        "query": query
    }
    print('sending request...')
    response = requests.post(
        f"{api_url}/search",
        headers=headers,
        json=payload
    )
    resp = [
        (i['title'], i['url']) for i in response.json()
    ]
    return resp

# 'insomnia'
for title, url in request('good sleep', [], []):
    display(HTML(f'<a href="{url}" target="_blank">{title}</a>'))

# Keyword search

In [244]:
def elastic_search(es_client, query, limit=3, filter=None, random=False):
    index_name = 'greenbro-content'
    if random:
        search_query = {
          "size": 10,
          "query": {
            "function_score": {
              "functions": [
                {
                  "random_score": {}
                }
              ]
            }
          }
        }
    else:
        search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']['hits'][:limit]

def pretty(search_results):
    result_docs = []

    include_fields = ['title', 'tags', 'relief', 'positive_effects', 'flavours']
    for hit in search_results:
        result_docs.append({k: v for k, v in  hit['_source'].items() if k in include_fields})
    return result_docs

try:
    query = 'Lemon, headache'
    
    res = elastic_search(es_client, query, random=True)
    sample = res[0]['_source']
    # print(sample.keys())
    print()
    print(pretty(res))
except ConnectionError:
    print('Elastic unreachable')


[{'title': 'OG 18', 'tags': 'Indica Dominant|70% Indica/30% Sativa', 'relief': 'arthritis chronic pain depression fibromyalgia headaches inflammation insomnia migraines', 'positive_effects': 'euphoria happy', 'flavours': 'diesel pine sweet'}, {'title': 'Peanut Butter Breath', 'tags': 'Hybrid,|50%/50%', 'relief': 'chronic pain depression loss of appetite nausea stress', 'positive_effects': 'body high hungry relaxing uplifting', 'flavours': 'herbal pine woody'}, {'title': 'Purple Sunset', 'tags': 'Indica Dominant|60% Indica/40% Sativa', 'relief': 'add/adhd anxiety depression stress', 'positive_effects': 'calming creative hungry relaxing uplifting', 'flavours': 'citrus fruity orange sour spicy sweet'}]


# Hybrid search

In [ ]:
import requests

def get_embed(query):
    url = "http://localhost:8000/embed"
    data = {"text": query}
    response = requests.post(url, json=data)
    if response.status_code == 200:
        res = np.array(json.loads(response.json()['embed']))
    else:
        print("Error:", response.status_code, response.text)
        raise RuntimeError
    return res

def elastic_hybrid_search(es_client, query, limit=3, filter=None, random=False):
    index_name = 'greenbro-content'
    vector_search_term = get_embed(query)
    knn_query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 100
    }
    
    response = es_client.search(
        index=index_name,
        query={
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["relief", "positive_effects", "flavours"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {  # TODO: add filter as parameter
                        "term": {
                            "category": "flower"
                        }
                    }
                }
        },
        knn=knn_query,
        size=5
    )
    return response['hits']['hits'][:limit]


try:
    query = 'Lemon, headache'
    
    res = elastic_hybrid_search(es_client, query, random=True)
    sample = res[0]['_source']
    #print(sample.keys())
    #print()
    print(pretty(res))
except ConnectionError:
    print('Elastic unreachable')

In [251]:
[{'score': i['_score'], 'res': i['_source']['item_name']} for i in res]

[{'score': 0.7898849, 'res': 'lemon-tree'},
 {'score': 0.77863276, 'res': 'golden-lemons'},
 {'score': 0.7349596, 'res': 'lemonhead'}]

In [ ]:


[{'score': scores[i], 'doc': self.documents[i]} for i in idx]

# Prepare data

In [211]:
import os
import pandas as pd

current_file_dir = os.getcwd()
root_data_dir = os.path.join(os.path.dirname(current_file_dir), 'data')
app_files_dir = os.path.join(root_data_dir, 'pipelines-data')

print(os.listdir(app_files_dir))

res_csv_path = os.path.join(app_files_dir, 'content_green_bro.csv')

content_db = pd.read_csv(res_csv_path)

['content_queries.json', 'golden_dataset.json', 'content_reviews_green_bro.json', 'models', 'api_db.csv', 'content_green_bro.csv', 'content_db_df.csv', 'ground_truth.json']


In [4]:
content_db.head()

,title,item_name,tags,relief,positive_effects,flavours,avg_rating,num_ratings
0,Casey Jones,casey-jones,Sativa Dominant|80% Sativa/20% Indica,add/adhd arthritis bipolar disorder chronic pa...,creative euphoria focus happy,citrus fruity lemon sweet,4.6,38
1,Gorilla Glue #4,gorilla-glue-4,Indica Dominant|60% Indica/40% Sativa,add/adhd bipolar disorder chronic pain depress...,body high euphoria happy uplifting,chemical chocolate coffee diesel pine sweet,4.6,339
2,Cherry Pie,cherry-pie,Indica Dominant|80% Indica/20% Sativa,add/adhd anxiety bipolar disorder chronic pain...,creative euphoria happy relaxing,berry cherry sweet,4.5,110
3,Amnesia Haze,amnesia-haze,Sativa Dominant|80% Sativa/20% Indica,add/adhd anxiety cancer chronic pain depressio...,creative euphoria giggly happy,citrus lemon sweet,4.5,55
4,Blueberry,blueberry,Indica Dominant|80% Indica/20% Sativa,add/adhd bipolar disorder depression insomnia ...,euphoria happy sleepy,berry blueberry sweet woody,4.5,95


Data for service

In [189]:
def add_url(postfix):
    url = os.path.join('https://www.allbud.com' + content_db_full.iloc[0]['link'])

content_db_full = pd.read_csv(os.path.join(app_files_dir, 'content_db_df.csv'))
content_db_full['url'] = content_db_full['link'].apply(lambda x: 'https://www.allbud.com' + x)
content_db = content_db_full[['url', 'item_name']].merge(content_db, on='item_name')

content_db.to_csv(os.path.join(app_files_dir, 'api_db.csv'), index=False)

In [192]:
def get_candidates(content_names_list):
    res = [
        {
            'title': row['title'], 'url': row['url'],
            'explanation': f"{row['tags']}: {row['positive_effects']}",
            'flavours': row['flavours']
        }
        for _, row in content_db[content_db['item_name'].isin(content_names_list)].iterrows()
    ]
    return res

content_names_list = ['blueberry', 'amnesia-haze', 'cherry-pie']

candidates = get_candidates(content_names_list)

candidates

[{'title': 'Cherry Pie',
  'url': 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/cherry-pie',
  'explanation': 'Indica Dominant|80% Indica/20% Sativa: creative euphoria happy relaxing',
  'flavours': 'berry cherry sweet'},
 {'title': 'Amnesia Haze',
  'url': 'https://www.allbud.com/marijuana-strains/sativa-dominant-hybrid/amnesia-haze',
  'explanation': 'Sativa Dominant|80% Sativa/20% Indica: creative euphoria giggly happy',
  'flavours': 'citrus lemon sweet'},
 {'title': 'Blueberry',
  'url': 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/blueberry',
  'explanation': 'Indica Dominant|80% Indica/20% Sativa: euphoria happy sleepy',
  'flavours': 'berry blueberry sweet woody'}]

In [204]:
os.listdir(app_files_dir)

['content_queries.json',
 'golden_dataset.json',
 'content_reviews_green_bro.json',
 'models',
 'api_db.csv',
 'content_green_bro.csv',
 'content_db_df.csv',
 'ground_truth.json']

In [217]:
models_dir = os.path.join(root_data_dir, 'pipelines-data', 'models')
index_file_path = os.path.join(models_dir, 'embeds_index.json')
embeds_file_path = os.path.join(models_dir, 'embeds.npy')
with open(index_file_path, 'r') as f:
    index = json.load(f)
embeds = np.load(embeds_file_path)
db = {}
for i, embed in enumerate(embeds):
    db[index[i]] = embed
print(db[index[-1]])

In [ ]:
class VectorDB:
    def __init__(self, index, embeddings):
        self.db = {}
        for i, embed in enumerate(embeddings):
            self.db[index[i]] = embed

    def get_item_vector(self, item_name):
        return self.db[item_name]


def get_vector_db(root_dir):
    models_dir = os.path.join(root_data_dir, 'pipelines-data', 'models')
    index_file_path = os.path.join(models_dir, 'embeds_index.json')
    embeds_file_path = os.path.join(models_dir, 'embeds.npy')
    with open(index_file_path, 'r') as f:
        index = json.load(f)
    embeds = np.load(embeds_file_path)
    print(embeds.shape, len(index))
    vector_db = VectorDB(index=index, embeddings=embeds)
    return vector_db

vector_db = get_vector_db(current_file_dir)
#vector_db.get_item_vector('gorilla-glue-4')

In [221]:
vector_db.db

{}

In [209]:
os.listdir('/Users/username/PycharmProjects/leaf-bro/jupyter_notebooks/data/pipelines-data/models')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/username/PycharmProjects/leaf-bro/jupyter_notebooks/data/pipelines-data/models'

In [201]:
import json
import datetime
import hashlib
import random

import numpy as np
import backoff
import openai
from openai import OpenAI


client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)


@backoff.on_exception(backoff.expo, openai.APIError)
@backoff.on_exception(backoff.expo, openai.RateLimitError)
@backoff.on_exception(backoff.expo,openai.Timeout)
@backoff.on_exception(backoff.expo, RuntimeError)
def gpt_query(gpt_params, verbose: bool = False, avoid_fuckup: bool = False) -> dict:
    print('connecting OpenAI...')
    if verbose:
        print(gpt_params["messages"][1]["content"])
    response = client.chat.completions.create(
        **gpt_params
    )
    gpt_response = response.choices[0].message.content
    if avoid_fuckup:
        if '[' in gpt_response or '?' in gpt_response or '{' in gpt_response:
            raise RuntimeError
    res = {'recs': gpt_response}
    res.update({'prompt_tokens': response.usage.completion_tokens, 'prompt_tokens': response.usage.prompt_tokens, 'total_tokens': response.usage.total_tokens})
    seed_phrase = f'{str(datetime.datetime.now().timestamp())}{gpt_response}'
    generation_id = str(hashlib.md5(seed_phrase.encode('utf-8')).hexdigest())[:12]
    res.update({'id': generation_id})
    return res

def random_shuffle(input_list):
    """
    my_list = ['apple', 'banana', 'cherry', 'date', 'fig']
    random_shuffle(my_list)
    """
    res = random.shuffle(input_list)
    return ', '.join(input_list)

def promt_generation(candidates):
  # TODO: use jinja2
  flavours_list = ['sweet', 'citrus', 'fruity', 'spicy', 'berry', 'pine', 'herbal', 'sour', 'lemon', 'woody', 'grape', 'tropical', 'diesel', 'skunky', 'nutty', 'blueberry', 'vanilla', 'creamy', 'candy', 'cherry']
  reliefs_list = ['depression', 'chronic pain', 'stress', 'insomnia', 'anxiety', 'fatigue', 'nausea', 'migraines', 'muscle spasms', 'headaches', 'ptsd', 'loss of appetite', 'add/adhd', 'inflammation', 'arthritis', 'bipolar disorder', 'cramps', 'pms', 'fibromyalgia', 'gastrointestinal disorder']
  promt = f"""
      generate 5 search query based on strain description below. All queries should be 3-5 words long. Do not include strain name.
      Do not add word "strain".
      Base query on flavours (not connstrained with this list): {random_shuffle(flavours_list)}
      reliefs: {random_shuffle(reliefs_list)}
      effects (body and mind)
      {candidates}
      Queries:
  """
  return promt

def promt_generation_2(candidates, query):
  promt = f"""
      Below you can find items with description in format `title: description`
      {candidates}
      Rerank items and return reranked item ids base on user query. Return only reranked items, comma-separate
      Do not add any explanation, just result
      User query: {query}
      expected result: [title, title, title]
      reranked:
  """
  return promt

def generate(gpt_prompt, verbose=False):
    gpt_params = {
        'model': 'gpt-3.5-turbo',
        'max_tokens': 500,
        'temperature': 0.7,
        'top_p': 0.5,
        'frequency_penalty': 0.5,
    }
    # gpt_promt = promt_generation(gen_candadates(db, ids))
    if verbose:
        print(gpt_promt)
    messages = [
        {
          "role": "system",
          "content": "You are a helpful assistant for medicine shopping",
        },
        {
          "role": "user",
          "content": gpt_prompt,
        },
    ]
    gpt_params.update({'messages': messages})
    res = gpt_query(gpt_params, verbose=False)
    return res


def load_reviews(reviews_file_path):
    with open(reviews_file_path, 'r') as f:
        reviews_dict = json.load(f)
    return reviews_dict

def get_item_reviews(item_name, reviews_dict):
    reviews = [i['review'].replace('\n', ' ') for i in reviews_dict if i['item_name'] == item_name]
    return reviews

def aggregate_reviews(content_df, reviews_dict):
    res = {}
    for name in content_df['item_name'].unique():
      res[name] = {'reviews': ''.join(get_item_reviews(name, reviews_dict))}
    return res

def test_generation(reviews_agregated, random_strain):
    prompt = promt_generation(reviews_agregated[random_strain]['reviews'])
    generated_result = generate(prompt)
    print(generated_result['recs'])
    print()
    reviews_agregated[random_strain]['reviews']

def test_reranking(candidates, query):
    [f"{i['title']}: {i['explanation']}, {i['flavours']}" for i in candidates]
    prompt = promt_generation_2(candidates, query)
    generated_result = generate(prompt)
    print(generated_result['recs'])

# reviews_dict = load_reviews(os.path.join(app_files_dir, 'content_reviews_green_bro.json'))
# reviews_agregated = aggregate_reviews(content_db, reviews_dict)
# # print(reviews_agregated['fire-og']['reviews'])

# random_strain = np.random.choice(list(reviews_agregated.keys()))
# test_generation(reviews_agregated, random_strain)

test_reranking(candidates, 'avoid cherry')

connecting OpenAI...
Blueberry, Amnesia Haze


In [203]:
a = []
a[0]

IndexError: list index out of range

Search index

In [71]:
key = list(reviews_agregated.keys())[0]
print(key)
reviews_agregated[key]

casey-jones


{'reviews': " First off, an elevation of my mood, a little bit of rejuvenation to my body, and even my thoughts, I noticed quickly while smoking CJ. I was feeling a little sluggish before I smoked this strain. No more. The taste has some earthiness to it, but mostly that sweet citrus taste. If you're fond of sativas I would recommend trying CJ. Smoke this pot.  I got an eighth of Casey Jones and it kicks some serious backside. I felt very refreshed taking a puff of this light green, fuzzy bud and got notes of tangerine, citrus and even a bit of watermelon. I highly recommend Casey Jones as a daytime strain to inspire creativity and socialization.  I didn't have the pleasure of growing her, but I bought a half oz of 'b' buds and squished it. What a sweet strain. Sweet as in sugar sweet. It came out of the press as foam mostly it was so sugary. Then I burned it 1 time and it turned Into beautiful thick amber plastic. For a low-level thc strain, I was very satisfied. Ty  This strain is ph

Generate ground trurh

In [66]:
from IPython.display import clear_output

def generate_and_save_ground_truth(input_dict, output_filename):
    for i in input_dict:
      reviews = input_dict[i]['reviews']
      prompt = promt_generation(reviews)
      input_dict[i]['queries'] = generate(prompt)
    with open(output_filename, 'w') as f:
      json.dump(input_dict, f)
    print('Data generated')
    return input_dict

clear_output()
output_filename = os.path.join(app_files_dir,'content_queries.json')
user_queries_dict = generate_and_save_ground_truth(reviews_agregated, output_filename)
user_queries_dict[random_strain]

{'reviews': " My first time with the Chapo, was a multi-high experience the flavor, omg…. The throat on a few hits reminded me of the drainage you’d get from decent quality “coke”, very sweet head high, spurts of clear thought, energy, then my favorite “Couch Lock”, and the world for now ceases.  Smoked a pre-roll of el chapo OG and fucking hell my eyes got sooo heavy and got baked asf!  I have a chronic disease this indica strain is a cure all fo me and I’m a long time smoker!!  I really enjoy this strains ability to reduce pain.  El Chapo OG Kush (Face/Off OG x SFV OG x OG Kush) by Boss Status Genetics/Cali Buds.Average Cannabinoid Content: THC: 10-23%CBD: 0.00-0.50%Terpene Profile:Myrcene, Limonene, Pinene, beta-Caryophyllene Medicinal Benefits include:pain relief gastrointestinal issues nausea relief insomnia relief relaxation appetite stimulationdepression relief  Superb! The best strain I've had in 2019 except that Presidential Kush. Grade A!  El Chapo OG (AKA: Shorty) is an Indi

In [ ]:

output_file_path = os.path.join(app_files_dir,'ground_truth.json')

In [68]:
#queries_dict = load_reviews(output_filename)
random_strain = np.random.choice(list(reviews_agregated.keys()))

print(queries_dict[random_strain]['queries']['recs'])

1. Fruity pain relief options
2. Energizing headache remedy
3. Relaxing stress relief choices
4. Uplifting mood support picks
5. Cerebral anxiety solutions


In [54]:
import re

def clean_text(text):
    cleaned_text = re.sub(r'^\d+\.\s*', '', text)
    return cleaned_text

def prepare_ground_truth(queries_dict, data_path):
    ground_truth = []
    
    for k in queries_dict:
        queries = queries_dict[k]['queries']['recs'].split('\n')
        ground_truth += [{'answer': k, 'query': clean_text(q)} for q in queries]
    print('Num generates: %d' % len(ground_truth))
    with open(data_path, 'w') as f:
      json.dump(ground_truth, f)
    print('data saved to %s' % data_path)
    return ground_truth
output_file_path = os.path.join(app_files_dir,'ground_truth.json')
ground_truth = prepare_ground_truth(queries_dict, output_file_path)

Num generates: 1455
data saved to /Users/username/PycharmProjects/leaf-bro/data/pipelines-data/ground_truth.json


In [56]:
import numpy as np

random_id = np.random.randint(0, len(ground_truth))
print(ground_truth[random_id])

queries_dict[ground_truth[random_id]['answer']]['reviews']

{'answer': 'gorilla-cookies', 'query': 'Citrus herbal pain relief'}


" Great high that is good for a night downtown that lasts for hours  This one hits me more like an Indica than a Sativa despite it being Sativa leaning. I get the heavy couch-locking feeling of the Gorilla Glue with the euphoric aspect (mentally and physically) from the GSC, along with that nice head buzz. While remaining clear-headed, I do find myself struggling to keep my eyes open after a good few dabs.  Awesome strain I was bouncing around getting things done Happy and care free Pretty dumb to be honest but didnt care  After consuming gorilla cookies thc oil in a vape device, numerous times, as a medical & recently recreational user, gorilla cookies is an extremely high sativa dominant smoke that in my case gives me plenty of nervous energy but no focus to get things done! Slight couch lock that's easily overcome for most, I personally didn't like the restless energy & feelings of anxiety gorilla cookies always produces. Your experience may differ!I didn't feel any other benefits s

# Project

find a strain and motivate user to buy it

In [76]:
def prepare_catalog(reviews_agregated):
    index = []
    corpus = []
    for key, content in reviews_agregated.items():
        index.append(key)
        corpus.append(content['reviews'])
    return index, corpus
index, corpus = prepare_catalog(reviews_agregated)
print(len(index), len(corpus))

291 291


In [87]:
models_dir = os.path.join(app_files_dir, 'models')

index_file_path = os.path.join(models_dir, 'embeds_index.json')
embeds_file_path = os.path.join(models_dir, 'embeds.npy')
with open(index_file_path, 'r') as f:
    index = json.load(f)
embeds = np.load(embeds_file_path)
print(embeds.shape, len(index))

(291, 768) 291


In [88]:
def get_pytorch_model(models_dir, model_name='multi-qa-distilbert-cos-v1'):
  from sentence_transformers import SentenceTransformer

  model_path = os.path.join(models_dir, model_name)

  if not os.path.exists(model_path):
      print('huggingface model loading...')
      embedder = SentenceTransformer(model_name)
      embedder.save(model_path)
  else:
      print('pretrained model loading...')
      embedder = SentenceTransformer(model_name_or_path=model_path)
  print('model loadind done')

  return embedder

class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [{'score': scores[i], 'doc': self.documents[i]} for i in idx]

embedder = get_pytorch_model(models_dir)
embedder

/Users/username/.pyenv/versions/3.10.13/envs/llmops-env/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


pretrained model loading...
model loadind done


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [89]:
search_engine = VectorSearchEngine(documents=index, embeddings=embeds)


In [91]:
user_question = 'citrus relaxation'

v = embedder.encode(user_question)

search_engine.search(v, num_results=5)

[{'score': 0.4901401, 'doc': 'lemon-tree'},
 {'score': 0.47702503, 'doc': 'golden-lemons'},
 {'score': 0.47451043, 'doc': 'hardcore-og'},
 {'score': 0.45465428, 'doc': 'orange-crush'},
 {'score': 0.4432276, 'doc': 'skywalker-og'}]

In [129]:
random_numbers = np.random.choice(np.arange(0, len(ground_truth) - 1), size=100, replace=False)
res = []
for random_id in random_numbers:
    origin = ground_truth[random_id]['answer']
    query = ground_truth[random_id]['query']
    v = embedder.encode(query)
    res.append( 1 if sum(origin==i['doc'] for i in search_engine.search(v, num_results=30)) > 0 else 0)

print(sum(res), len(res))

39 100


In [131]:
[ground_truth[i] for i in random_numbers[np.where(res==1)]]


/var/folders/6_/c5005cfj5_s0k2v7t69q1qsh0000gn/T/ipykernel_39617/649090524.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  [ground_truth[i] for i in random_numbers[np.where(res==1)]]


[]

In [166]:
def eval_hitrate(golden_dataset):
    res = []
    for entry in golden_dataset:
        v = embedder.encode(entry['query'])
        res.append( 1 if sum(entry['origin']==i['doc'] for i in search_engine.search(v, num_results=30)) > 0 else 0)
    print('Hit rate: %.3f, num entries: %d' % (sum(res)/len(res), len(res)))

def generate_golden_set():
    items_list = list(set([i['answer'] for i in ground_truth]))
    random.shuffle(items_list)
    random.shuffle(ground_truth)
    result = []
    test_queries = []
    for i in items_list[:100]:
        result.append(i)
        test_queries.append([j['query'] for j in ground_truth if j['answer']==i][0])
    golden_dataset = [{'origin': origin, 'query': query} for origin, query in zip(result, test_queries)]
    return golden_dataset

data_path = os.path.join(app_files_dir, 'golden_dataset.json')
golden_dataset = [{'origin': origin, 'query': query} for origin, query in zip(result, test_queries)]
with open(data_path, 'w') as f:
  json.dump(golden_dataset, f)

In [160]:
[i['answer'] for i in np.array(ground_truth)[np.where(np.array(res)==1)]]

['alaskan-thunder-fuck',
 'key-lime-pie',
 'la-confidential',
 'grape-pie',
 'monster-cookies',
 'white-og',
 'purple-punch',
 'island-sweet-skunk',
 'gobbstopper',
 'gelato-cake',
 'white-widow',
 'chocolate-diesel',
 'lavender-jones',
 'sour-berry',
 '101-headband',
 'super-boof',
 'sfv-og',
 'cherry-pie',
 '101-headband',
 'chocolate-diesel',
 'kosher-kush',
 'granddaddy-purple',
 'master-kush',
 'zoap',
 'papaya-cake',
 'tahoe-og',
 'sour-tangie',
 'lemon-skunk',
 'slurricane',
 'zkittlez-cake']

In [163]:
# queries_dict['skywalker-og']['reviews']